# **Design Space Exploration**
- Define the most suitable hardware architecture to implement StereoSpike network.
    - Leverage the use of Depthwise Separable (DWS) convolutions.

## **Conventional Architectures**
Based on [1], three main hardware architectures have been proposed to implement DWS convolutions:
1. *Unified Engine* (UE): single processing element (PE) array for sequential execution.
2. *Separated Engine* (SE): dedicated PE arrays for parallel execution.
3. *Reconfigurable Engine* (RE): single PE array with dinamic partitioning for parallel execution.

![](./images/conventional_arch.png)

## 1. Libraries

In [1]:
import os

import matplotlib.pyplot as plt
import matplotlib.font_manager as fm
from matplotlib import rcParams
import matplotlib.patches as mpatches
from mpl_toolkits.mplot3d import Axes3D 
import numpy as np

import math
from dataclasses import dataclass
from typing import List, Tuple, Any, Optional
from pprint import pprint

import seaborn as sns
import pandas as pd

## 2. Functions

# **References**
[1] B. Li et al., “Dynamic Dataflow Scheduling and Computation Mapping Techniques for Efficient Depthwise Separable Convolution Acceleration,” IEEE Trans. Circuits Syst. I: Reg. Papers, vol. 68, no. 8, pp. 3279–3292, Aug. 2021. https://doi.org/10.1109/TCSI.2021.3078541